In [2]:
from matplotlib import pyplot as plt
import numpy as np

In [3]:
N = 5*3
x = 4

#returns x^r mod N
def period_finding(N, x):
    
    return 
    
    #period_found = False
    
    #while not period_found
        

rs = np.arange(200)

plt.plot(rs, period_finding(N,x,rs))

period_finding(N,x,rs)

SyntaxError: invalid syntax (<ipython-input-3-392e42814a2e>, line 8)